# Capstone project Week 3

In [279]:
import requests
import pandas as pd

#### Let's get the data using requests package and `get` http call

In [189]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
response.content

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":867606113,"wgRevisionId":867606113,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat

In [ ]:
#### I am going to use lxml for parsing the html

In [190]:
import lxml.html as lh
xml = lh.fromstring(response.content)

In [ ]:
#### I am going to find all tables on that wiki page and then select the first table.
#### Assumption here is that first table is always the data. 

In [191]:
table = xml.findall(".//*/table")

In [280]:
#### Get all trs from the table
#### which has header and rows
#### we then get contents from all those rows and create a list of lists

In [192]:
header, *rows = table[0].xpath("tbody/tr")
hdr = []
for h in header.xpath("th"):
    hdr.append(h.text_content().strip())

newrows = []
    
for row in rows:
    x = []
    for column in row.xpath("td"):
        x.append(column.text_content().strip())
    newrows.append(x)
newrows 


[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 

In [281]:
#### we then create a dataframe from the result above and process it as requested.
#### 1. We filter out all the rows those has Borough is 'Not assigned'
#### 2. Then those with Neighbourhood not assigned we put Borough as Neighbourhood
#### 3. Then we group by Postcode and concatenate all Neighbourhoods (comma separated)

In [245]:
dataframe = pd.DataFrame(newrows)
dataframe.columns = hdr
dataframe.head()
filtered  = dataframe[dataframe.Borough != "Not assigned"]
def handle_non_assigned(row):
#    print(row)
    if(row.Neighbourhood == "Not assigned"):
        row["Neighbourhood"] = row["Borough"]
        return row
    else:
        return row
    
filtered = filtered.apply(handle_non_assigned, axis=1)
filtered
x = filtered.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(lambda x: ",".join(x)).reset_index()


In [ ]:
#### We get 103 rows in total

In [250]:
x.shape

(103, 3)